# เวิร์คชอปจากลิงก์ https://github.com/yohman/workshop-remote-sensing
# วีดีโอ https://www.youtube.com/watch?v=gi4UdFsayoM
# การใช้งาน GEE ใน jupyter https://book.geemap.org/chapters/01_introduction.html

In [1]:
# the regulars
import pandas as pd
import geopandas as gpd

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfJohXkvJEHJax4qa9IAftw-HTB6-vu7kQVROu9IkYjvJ6XY2XeJrQdqcHA



Successfully saved authorization token.


In [3]:
# coordinates of the Camp Fire
lat =  16.813374671902526
lon =  100.26223196853726

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2023-01-01'

# end date
end_date = '2023-03-31'

In [4]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [5]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 6


In [6]:
# information about the first image in our collection
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7581, 7731],
   'crs': 'EPSG:32647',
   'crs_transform': [30, 0, 478485, 0, -30, 2033415]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7581, 7731],
   'crs': 'EPSG:32647',
   'crs_transform': [30, 0, 478485, 0, -30, 2033415]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7581, 7731],
   'crs': 'EPSG:32647',
   'crs_transform': [30, 0, 478485, 0, -30, 2033415]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7581, 7731],
   'crs': 'EPSG:32647',
   'crs_transform': [30, 0, 478485, 0, -30, 2033415]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
 

In [7]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()

19.22

In [8]:
# when was this image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2023-01-01'

In [9]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [10]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

In [11]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [12]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)
    
    # display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date,cloud]

    # append the data 
    data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2023-01-01 Cloud cover: 19.22


Image # 1 2023-01-17 Cloud cover: 6.93


Image # 2 2023-02-02 Cloud cover: 1.44


Image # 3 2023-02-18 Cloud cover: 87.93


Image # 4 2023-03-06 Cloud cover: 0.54


Image # 5 2023-03-22 Cloud cover: 0.42


In [13]:
# our data in a dataframe
df

,Image #,Date,Cloud Cover
0,0,2023-01-01,19.22
1,1,2023-01-17,6.93
2,2,2023-02-02,1.44
3,3,2023-02-18,87.93
4,4,2023-03-06,0.54
5,5,2023-03-22,0.42


In [14]:
# create a list of images we want (before, during, after)
landsat_sequence = [0,2,5]

In [15]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

In [16]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region':roi
             }

In [17]:
for i in landsat_sequence:
    
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 0 2023-01-01 Cloud cover: 19.22


Image # 2 2023-02-02 Cloud cover: 1.44


Image # 5 2023-03-22 Cloud cover: 0.42


In [18]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [19]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 0 2023-01-01


Image # 2 2023-02-02


Image # 5 2023-03-22


In [20]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
m

In [21]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [22]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

In [23]:
my_map.save('camp.html')